In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.utils import _down_cast, data_preprocessing, diff_lists, log_status, ensemble_submissions_uncertainty
from utils.configure_logger import configure_logger
from utils.utils import prefixes_in_column
from utils import constants

configure_logger()
from logging import getLogger
logger = getLogger(__name__)

import warnings
warnings.simplefilter("ignore")

In [27]:
DATA_BASE_PATH = constants.DATA_BASE_PATH
DATA_BASE_PATH_UNCERTAINTY = constants.DATA_BASE_PATH_UNCERTAINTY
SALES_EVALUATION = constants.SALES_EVALUATION 
SALES_VALIDATION = constants.SALES_VALIDATION
CALENDAR = constants.CALENDAR 
SAMPLE_SUBMISSION = constants.SAMPLE_SUBMISSION 
SELL_PRICES = constants.SELL_PRICES

PRECOMPUTED_BASE_PATH = constants.PRECOMPUTED_BASE_PATH #'../data/uncertainty/features/'

DAYS: int = constants.DAYS #28
QUANTILES: int = constants.QUANTILES 

AGG_LEVEL_COLUMNS = constants.AGG_LEVEL_COLUMNS
D_CROSS_VAL_START_LIST = constants.D_CROSS_VAL_START_LIST

# to simple get the precomputed name
precomputed_name = lambda store, eval_val: f'processed_{store}_{eval_val}.pkl'

TEST_PATH = constants.TEST_PATH#'test/'
PREDICTION_BASE_PATH = constants.PREDICTION_BASE_PATH
SUBMISSION_BASE_PATH = constants.SUBMISSION_BASE_PATH

SUB_D_START_VAL: int = constants.SUB_D_START_VAL
SUB_D_START_EVAL: int = constants.SUB_D_START_EVAL

# the columns are always included after feature processing
# because they are required in the training and submission format
DROP_FEATURE_COLUMNS: list = constants.DROP_FEATURE_COLUMNS #['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'd', 'sold']

In [3]:
def read_concat_predictions(fold_name: int, exclude_columns: list = [], include_columns: list = [], sparse = False, use_all = False, load_submissions_path: str = 'temp_submissions/'):
    """ 
    For specified fold, read the predictions for all aggregation levels 
    and stack them together in one dataframe.
    """
    # D_CV_START_LIST
    # if fold_name not in D_CV_START_LIST:
        # raise ValueError('fold_name must be a value in D_CV_START_LIST')
        
    exclude_columns = '_'.join(exclude_columns)
    if exclude_columns == '':
        exclude_columns = 'None'

    logger.info('loading files under path:' + f'../data/uncertainty/fold_{fold_name}/' + load_submissions_path)

    TEST_NUMB = 0
    TEST_NUMBER = 9

    dfs: list = []
    for level in list(AGG_LEVEL_COLUMNS.keys())[TEST_NUMB:TEST_NUMBER]:
        agg_columns = AGG_LEVEL_COLUMNS[level]
        group_names = '_'.join(agg_columns)
        if group_names == '':
            group_names = 'Total_X'
        
        file_path = f'../data/uncertainty/fold_{str(fold_name)}/' + load_submissions_path 
        file_path += f'lgb_val_nt_{group_names}_'
        if use_all:
            file_path += f'use_all.csv'  
        elif include_columns == None:
            file_path += f'exclude_{"_".join(exclude_columns)}.csv'            
        elif isinstance(include_columns, list):
            file_path += f'include_{"_".join(include_columns)}.csv'
        
        dfs.append(file_path)
    dfs = ensemble_submissions_uncertainty(dfs)
    dfs['d_cv_start'] = fold_name
    return dfs

In [4]:
def merge_predictions_with_true_sales_and_weights(df: pd.DataFrame, df_sub1: pd.DataFrame, df_sub2: pd.DataFrame, weights: pd.DataFrame):
    # 
    df_sub2 = df_sub2.drop('d_cv_start', axis=1)
    
    # to be able to merge
    df_sub1['id_merge'] = df_sub1['id']\
        .apply(lambda x: x.split('.')[0])
    df_sub1['quantile'] = df_sub1['id']\
        .apply(
            lambda x: float(
                '.'.join([
                x.split('.')[-2], 
                x.split('.')[-1].split('_')[0]
                ])
            )
        )
    df_sub2['id_merge'] = df_sub2['id']\
        .apply(lambda x: x.split('.')[0])
    df_sub2['quantile'] = df_sub2['id']\
        .apply(
            lambda x: float(
                '.'.join([
                x.split('.')[-2], 
                x.split('.')[-1].split('_')[0]
                ])
            )
        )
    # merge both submissiosn
    df_sub = pd.merge(
        df_sub1,
        df_sub2,
        how = 'inner',
        on = ['d', 'id_merge', 'quantile', 'id']
    )

    # merge true sales on prediction df
    p = pd.merge(
        df,
        df_sub,
        how='right',
        on=['id_merge', 'd',]
    )
    # del df; del df_sub_val
    p['id_merge'] = p['id_merge'].astype(str)

    for c in ['sold', 'revenue']:
        p[c] = p[c].astype(np.float32)
        
        
    # merge weights on it
    p = pd.merge(
        p,
        weights,
        how = 'left',
        on = ['id_merge', 'd_cv_start']
    )
    return p

In [29]:
def load_all_weights():
    """ Load the weights for each fold and return the stacked dataframe """
    dfs = []
    for d_cv_start in D_CROSS_VAL_START_LIST:    
        # load weights
        weights = pd.read_csv(f'../data/uncertainty/fold_{d_cv_start}/weights_validation.csv')
        weights['d_cv_start'] = d_cv_start
        weights['id_merge'] = weights['Agg_Level_1'] + '_' + weights['Agg_Level_2']
        dfs.append(weights)
    return pd.concat(dfs).reset_index(drop=True).drop('Unnamed: 0', axis=1)

In [32]:
def get_scale(d: pd.DataFrame, d_cv_start: int):
    idx = d['d_temp'] < d_cv_start
    d = d[idx]
    am = lambda x: x.diff().abs().mean(skipna=True)
    df_scale = d.groupby(['id_merge']).agg(
        {
            'sold': am,
        } 
        ).reset_index(drop=False)
    df_scale = df_scale.rename({'sold': 'scale'}, axis=1)
    return df_scale

In [5]:
# define experiments
EXPERIMENTS_DICT = {
    "seasonal": {
        "BASE": [],
        "INCLUDE_COLUMNS_LIST": [
            ['auto_sold_ewm'],
            ['seasonal_weekday','auto_sold_ewm'],
            ['seasonal_monthday','auto_sold_ewm'],
            ['seasonal_weekday','seasonal_monthday','auto_sold_ewm'],
            ['seasonal','auto_sold_ewm'],
        ]
    },
    "state vs. store": {
        "BASE": ['seasonal', 'auto_sold_ma'],
        "INCLUDE_COLUMNS_LIST": [
            [],
            ['state_id',],
            ['store_id',],
            ['state_id', 'store_id']
        ]
    },
    "ewm vs. ma": {
        "BASE": ['seasonal'],
        "INCLUDE_COLUMNS_LIST": [
            ['auto_sold_ewm'],
            ['auto_sold_ma'],
            ['auto_sold_ewm', 'auto_sold_ma'],
        ]
    },
    "quantiles vs. std": {
        "BASE": ['seasonal', 'auto_sold_ma'],
        "INCLUDE_COLUMNS_LIST": [
            [],
            ['auto_sold_qtile'],
            ['auto_sold_std'],
            ['auto_sold_qtile','auto_sold_std'],   
        ]
    },
    "price auto/momentum": {
        "BASE": ['seasonal', 'auto_sold_ma'],
        "INCLUDE_COLUMNS_LIST": [
            [],
            ['price_auto_std'],
            ['price_momentum'],
            ['price_uncond'],
            ['price_auto_std', 'price_momentum'],
            ['price_auto_std', 'price_momentum', 'price_uncond']
        ]
    },
    "best models": {
        "BASE": ['seasonal', 'auto_sold_ma'],
        "INCLUDE_COLUMNS_LIST": [
            ['state_id', 'store_id'],
            ['auto_sold_std', 'state_id', 'store_id'],
        ]
    },
    "full vs. sparse ma" : {
        "BASE": ['seasonal'],
        "INCLUDE_COLUMNS_LIST": [
            ['auto_sold_ma', 'auto_sold_std', 'auto_sold_qtile', 'auto_sold_ewm', 'state_id', 'store_id'],
            ['auto_sold_ma_28', 'auto_sold_ma_56', 'auto_sold_ma_168', 'state_id', 'store_id']
        ]
    },
    "sparse vs. kbest": {
        "BASE": ['seasonal', 'state_id', 'store_id'],
        "INCLUDE_COLUMNS_LIST": [
            ['auto_sold', 'price', 'kbest'],
            ['auto_sold_ewm_112', 'auto_sold_ewm_28',
             'auto_sold_qtile_28_0.5', 'auto_sold_ma_28', 
             'auto_sold_qtile_28_0.9',],
        ]
    },
    'full vs. sparse': {
        "BASE": ['seasonal'],
        "INCLUDE_COLUMNS_LIST": [
            ['auto_sold_ma', 'auto_sold_std', 'auto_sold_qtile', 'auto_sold_ewm', 'state_id', 'store_id'],
            # ['auto_sold_ma_28', 'auto_sold_ma_56', 'auto_sold_ma_168', 'state_id', 'store_id'],
            ['auto_sold_std_3', 'auto_sold_std_56', 'auto_sold_std_168', 
            'auto_sold_ma_7',  'auto_sold_ma_28', 'auto_sold_ma_56', 
            'auto_sold_qtile_28_0.25', 'auto_sold_qtile_168_0.25', 'auto_sold_qtile_56_0.1', 
            'state_id', 'store_id'],
            ['state_id', 'store_id', 'auto_sold_ewm_112', 'auto_sold_ewm_28',
             'auto_sold_qtile_28_0.5', 'auto_sold_ma_28', 
             'auto_sold_qtile_28_0.9',],
        ]
    },
}

In [31]:
# load true sales values
# these variables are used later on
FORCE_RELOAD = False
try:
    # simple code to check if variable exists
    d_int + 1
    if FORCE_RELOAD:
        raise Exception()
except:
    # if not, load again
    # takes about 2-3 minutes to reload and parse
    # not the most beautiful method but it works
    d = pd.read_parquet('../data/uncertainty/cv_template/temp.parquet')
    try:
        d_int = pd.read_parquet('../data/uncertainty/cv_template/temp_d_int.parquet')['d_int']
    except:
        d_int = d['d'].apply(lambda x: int(x.split('_')[1]))
        d_int.to_frame('d_int').to_parquet('../data/uncertainty/cv_template/temp_d_int.parquet', index = False)
print(d_int.max())   
d['d_temp'] = d_int
d.head(5)

1969


,Level,agg_column1,agg_column2,d,sold,revenue,id_merge,d_temp
0,Level1,Total,X,d_10,24858.0,63029.78,Total_X,10
1,Level1,Total,X,d_100,23653.0,65665.71,Total_X,100
2,Level1,Total,X,d_1000,29241.0,82351.45,Total_X,1000
3,Level1,Total,X,d_1001,33804.0,93975.55,Total_X,1001
4,Level1,Total,X,d_1002,42447.0,118961.96,Total_X,1002


In [9]:
# compute scales for individual series
dfs = []
for d_cv_start in D_CROSS_VAL_START_LIST:
    scale = get_scale(d, d_cv_start)
    scale['d_cv_start'] = d_cv_start
    dfs.append(scale)
scales = pd.concat(dfs).reset_index(drop=True)
scales.head(5)

In [64]:
# select experiment
# experiment_spec = EXPERIMENTS_DICT['sparse vs. kbest']
experiment_spec = EXPERIMENTS_DICT['ewm vs. ma']
include_columns_list = [experiment_spec['BASE'] + include_columns
    for include_columns in experiment_spec['INCLUDE_COLUMNS_LIST']]
for i, include_columns in enumerate(include_columns_list):
    if 'kbest' in include_columns:
        include_columns_list[i] = ['k_best']

# load 2 submissions
print('first model: ' + str(include_columns_list[0]))
include_columns = include_columns_list[1]
dfs = [
    read_concat_predictions(d_cv_start, exclude_columns=[], include_columns=include_columns) 
    for d_cv_start in D_CROSS_VAL_START_LIST
]
sub1 = pd.concat(dfs).reset_index(drop=True)
print('second model: ' + str(include_columns_list[1]))
include_columns = include_columns_list[0]
dfs = [
    read_concat_predictions(d_cv_start, exclude_columns=[], include_columns=include_columns) 
    for d_cv_start in D_CROSS_VAL_START_LIST
]
sub2 = pd.concat(dfs).reset_index(drop=True)

# weights
weights = load_all_weights()

# merge all files together
final_df = merge_predictions_with_true_sales_and_weights(d, sub1, sub2, weights)


# merge scales also
final_df = pd.merge(
    final_df,
    scales,
    how = 'left',
    on = ['id_merge', 'd_cv_start']
)

2023-12-28 23:50:37 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1802/temp_submissions/
2023-12-28 23:50:37 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1830/temp_submissions/
2023-12-28 23:50:37 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1858/temp_submissions/
2023-12-28 23:50:37 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1886/temp_submissions/


first model: ['seasonal', 'auto_sold_ewm']


2023-12-28 23:50:37 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1914/temp_submissions/
2023-12-28 23:50:38 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1802/temp_submissions/
2023-12-28 23:50:38 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1830/temp_submissions/
2023-12-28 23:50:38 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1858/temp_submissions/
2023-12-28 23:50:38 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1886/temp_submissions/


second model: ['seasonal', 'auto_sold_ma']


2023-12-28 23:50:38 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1914/temp_submissions/


In [163]:
def DM_test_pinball(df, h, p_crit: float = 0.05, one_sided = False):
    """ 
    Test the two-sided significance of 2 quantile forecasts by the DM test
    The following steps are performed
    - for predictions x and y, compute the pinball residual
    - take their difference
    - normalize the residuals by the series volatility (scaled pinball loss)
    - average the residuals by quantiles
    - average the avg. residuals by relative revenue per aggregation level
    - average all aggregation level unweighted
    - >> this leaves a univariate time series, for t=1,..,28
    - compute diebold mariano test
    
    If one_sided = True, the null-hypothesis is that model 1 does predict equally well as model 2
    The alternative, is that model 1 predicts better than model 2
    """
    quantile = df['quantile']
    #
    resid_x = df['sold'] - df['pred_x']
    idx = resid_x >= 0
    pinball_resid_x = resid_x
    pinball_resid_x[idx] = resid_x[idx] * quantile[idx]
    pinball_resid_x[~idx] = resid_x[~idx] * (quantile[~idx]-1)
    #
    resid_y = df['sold'] - df['pred_y']
    idx = resid_y >= 0
    pinball_resid_y = resid_y
    pinball_resid_y[idx] = resid_y[idx] * quantile[idx]
    pinball_resid_y[~idx] = resid_y[~idx] * (quantile[~idx]-1)
    #
    df['pinball_resid'] = pinball_resid_x - pinball_resid_y
    #
    
    # TODO: SCALE THE SERIES BY HISTORIC VOLATILITY
    df['pinball_resid'] = df['pinball_resid'] / df['scale']
    #########################
    
    if (pinball_resid_y < 0).sum() > 0 or (pinball_resid_x < 0).sum() > 0:
        raise Exception('negative residuals, something went wrong')
    
    agg_dict = {
        'revenue': 'last',
        'pinball_resid': 'mean',
        'Level': 'last',
        'Weight': 'last'
    }
    # aggregate all quantiles for each series individually
    df_qtile_avg = df.groupby(['d', 'id_merge']).agg(agg_dict).reset_index(drop=False)
    
    # compute the weighted mean per level
    wm = lambda x: np.average(x, weights=df.loc[x.index, 'Weight'])
    agg_dict = {
        'pinball_resid': wm,
    }
    df_d_level_avg = df_qtile_avg.groupby(['d', 'Level']).agg(agg_dict).reset_index(drop=False)
    
    # compute the daily mean over all aggregation levels
    df_d_avg = df_d_level_avg.groupby(['d']).agg({'pinball_resid': 'mean'}).reset_index(drop=False)
    
    # compute cov
    p_s = df_d_avg['pinball_resid']
    # return p_s
    mean = p_s.mean()
    T = len(p_s)
    
    def auto_cov(resid, lag, mean):
        resid = list(resid)
        cov = 0
        T = float(len(resid))
        for i in np.arange(0, len(resid)-lag):
            cov += ((resid[i+lag])-mean)*(resid[i]-mean)
        return (1/(T))*cov
    
    gamma = []
    for lag in range(h):
        gamma.append(auto_cov(p_s, lag, mean))
    
    # compute stat
    V_d = (gamma[0] + 2*sum(gamma[1:]))/T
    DM_stat=V_d**(-0.5)*mean
    harvey_adj=( ( T+1-2*h+h*(h-1)/T) / T ) ** 0.5
    # print(harvey_adj)
    DM_stat = harvey_adj*DM_stat

    # compute p_value
    from scipy.stats import t
    # if one sided
    if one_sided:
        p_value = t.cdf(DM_stat, df = T - 1)
    else:
        p_value = 2*t.cdf(-abs(DM_stat), df = T - 1)
    h0_rejected = True if p_value < p_crit else (True if pd.isna(p_value) else False)
    return DM_stat, round(p_value,5), h0_rejected

In [164]:
# r = DM_test_pinball(final_df, 9, p_crit=0.1)
# r.plot()
# plt.grid()
# plt.show()
# from statsmodels.graphics.tsaplots import plot_pacf
# plot_pacf(r.diff().dropna())

In [169]:
r = DM_test_pinball(final_df, 9, p_crit=0.1, one_sided=True)
# fd = final_df.copy()
# fd['ids'] = fd['id_merge']
# group_by = ['ids']
print(f"mean_stat: {r[0]} - rejected: {r[2]} - p value: {r[1]}")

mean_stat: -1.330041895963647 - rejected: True - p value: 0.09284


In [171]:
def perform_full_DM_test(include_columns_list):
    # load 2 submissions
    include_columns = include_columns_list[0]
    dfs = [
    read_concat_predictions(d_cv_start, exclude_columns=[], include_columns=include_columns) 
    for d_cv_start in D_CROSS_VAL_START_LIST
    ]
    sub1 = pd.concat(dfs).reset_index(drop=True)
    
    include_columns = include_columns_list[1]
    dfs = [
    read_concat_predictions(d_cv_start, exclude_columns=[], include_columns=include_columns) 
    for d_cv_start in D_CROSS_VAL_START_LIST
    ]
    sub2 = pd.concat(dfs).reset_index(drop=True)

    # weights
    weights = load_all_weights()

    # merge all files together
    final_df = merge_predictions_with_true_sales_and_weights(d, sub1, sub2, weights)
    
    # merge scales also
    final_df = pd.merge(
        final_df,
        scales,
        how = 'left',
        on = ['id_merge', 'd_cv_start']
    )

    return DM_test_pinball(final_df, 5, p_crit=0.1, one_sided = True)

In [220]:
res = {}
res2 = {}
for experiment_name, experiment_spec in EXPERIMENTS_DICT.items():
    res[experiment_name] = {}
    base = experiment_spec['BASE']
    include_columns_list = [base + include_columns for include_columns in experiment_spec['INCLUDE_COLUMNS_LIST']]
    for i, include_columns in enumerate(include_columns_list):
            if 'kbest' in include_columns:
                    include_columns_list[i] = ['k_best']

    import itertools
    df = pd.DataFrame({})
    df.index = df.index.astype('str')
    for incl in itertools.combinations(include_columns_list,2):
        r = perform_full_DM_test(incl)
        name = [' '.join(i) for i in incl]
        n0 = name[0].replace((' '.join(base) + ' ') if len(base) > 0 else '', '')
        n1 = name[1].replace((' '.join(base) + ' ') if len(base) > 0 else '', '')
        if n0 == ' '.join(base):
            n0 = 'base'
        if n1 == ' '.join(base):
            n1 = 'base'
        df.loc[n0, n1] = f'{round(r[0],3)} ({round(r[1],3)})'
        df.loc[n1, n0] = f'{round(-r[0],3)} ({round(1-r[1],3)})'
        name = ' == '.join(name)
        res[experiment_name][name] = (r[1], r[0])
    df = df.fillna('X')
    res2[experiment_name] = df
res

2024-01-02 10:24:51 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1802/temp_submissions/
2024-01-02 10:24:51 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1830/temp_submissions/
2024-01-02 10:24:51 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1858/temp_submissions/
2024-01-02 10:24:52 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1886/temp_submissions/
2024-01-02 10:24:52 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1914/temp_submissions/
2024-01-02 10:24:52 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1802/temp_submissions/
2024-01-02 10:24:52 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1830/temp_submissions/
2024-01-02 10:24:52 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1858/temp_submissions/
2024-01-02 10:24:52 - __main__ - INFO - loading files under path:../data/uncertainty/fold_1886/t

{'seasonal': {'auto_sold_ewm == seasonal_weekday auto_sold_ewm': (1.0,
   22.818979914211234),
  'auto_sold_ewm == seasonal_monthday auto_sold_ewm': (0.84375,
   1.0136806408021153),
  'auto_sold_ewm == seasonal_weekday seasonal_monthday auto_sold_ewm': (1.0,
   21.38830242441285),
  'auto_sold_ewm == seasonal auto_sold_ewm': (1.0, 47.687629213826646),
  'seasonal_weekday auto_sold_ewm == seasonal_monthday auto_sold_ewm': (0.0,
   -9.636306974031259),
  'seasonal_weekday auto_sold_ewm == seasonal_weekday seasonal_monthday auto_sold_ewm': (0.95078,
   1.663606342479339),
  'seasonal_weekday auto_sold_ewm == seasonal auto_sold_ewm': (0.99876,
   3.080907919417292),
  'seasonal_monthday auto_sold_ewm == seasonal_weekday seasonal_monthday auto_sold_ewm': (1.0,
   14.83588007875148),
  'seasonal_monthday auto_sold_ewm == seasonal auto_sold_ewm': (1.0,
   18.537154759876827),
  'seasonal_weekday seasonal_monthday auto_sold_ewm == seasonal auto_sold_ewm': (0.99992,
   3.8724022000049723)},
 '

In [244]:
drop_dict = {
    'seasonal': 'auto_sold_ewm',
}
pd.set_option('display.max_colwidth', 1000)  # None means unlimited column width

for key, item in res2.items():
    if key in drop_dict:
        df: pd.DataFrame = res2[key]
        df.index = df.index.str.replace(drop_dict[key],'base')
        df.columns = df.columns.str.replace(drop_dict[key],'base')
        
        df.index = df.index.str.replace(' base', '')
        df.columns = df.columns.str.replace(' base', '')        
        res2[key] = df
        
    # order the table with X diagonally
    df = res2[key]
    df.index.name = 'model 1'
    df.columns.name = 'model 2'
    res2[key] = df[list(df.index)]
        
for key, item in res2.items():
    res2[key]: pd.DataFrame
    # n = key.replace(' ', '_')
    file_name = key.replace('/', '_')
    # res2[key].to_latex(f'../tables/{n}.tex', index_names = True)

    df = res2[key]
    tab = res2[key].to_latex(index_names = True)
    latex_table_lines = tab.split('\n')
    p_crit = 0.05
    # for line in latex_table_lines:
    #     print(line)
    for k, (i, row) in enumerate(df.iterrows()):
        # Iterate over columns
        for j, col in enumerate(df.columns):
            if i == col:
                continue
            if float(row[col].split('(')[1].removesuffix(')')) < p_crit:
                latex_table_lines[k+5] = latex_table_lines[k+5].replace(str(row[col]), '\\cellcolor{green}' + f' {row[col]}')
        
    latex_table = '\n'.join(latex_table_lines)
    with open(f'../tables/{file_name}.tex', 'w') as file:
        file.write(latex_table)

list(res2.items())[4][1]

model 2,base,price_auto_std,price_momentum,price_uncond,price_auto_std price_momentum,price_auto_std price_momentum price_uncond
model 1,,,,,,
base,X,0.434 (0.668),-3.536 (0.0),1.203 (0.884),0.003 (0.501),0.771 (0.779)
price_auto_std,-0.434 (0.332),X,-1.055 (0.147),0.401 (0.655),-1.619 (0.054),0.197 (0.578)
price_momentum,3.536 (1.0),1.055 (0.853),X,2.155 (0.984),0.737 (0.769),1.58 (0.942)
price_uncond,-1.203 (0.116),-0.401 (0.345),-2.155 (0.016),X,-0.725 (0.235),-0.457 (0.324)
price_auto_std price_momentum,-0.003 (0.499),1.619 (0.946),-0.737 (0.231),0.725 (0.765),X,1.015 (0.844)
price_auto_std price_momentum price_uncond,-0.771 (0.221),-0.197 (0.422),-1.58 (0.058),0.457 (0.676),-1.015 (0.156),X


In [ ]:
for experiment_name, experiment_results in res.items():
    print(experiment_name)
    for key, item in experiment_results.items():
        if item[0] <= 1:
            print(key, item)

In [100]:
pd.set_option('display.max_colwidth', 1000)  # None means unlimited column width
for key, a_dict in res.items():
    a = {}
    print('-----------------------------------------')
    print(key)
    for k,v in a_dict.items():
        a[k] = [v[0]]
    print(pd.DataFrame(a).set_index(pd.Index(['p_value'])).transpose())
    print('-----------------------------------------')
    
pd.set_option('display.max_colwidth', 1000)  # None means unlimited column width
for key, a_dict in res.items():
    a = {}
    print('-----------------------------------------')
    print(key)
    for k,v in a_dict.items():
        a[k] = [v[1]]
    print(pd.DataFrame(a).set_index(pd.Index(['stat'])).transpose())
    print('-----------------------------------------')

-----------------------------------------
seasonal
                                                                                     p_value
auto_sold_ewm == seasonal_weekday_auto_sold_ewm                                      0.00000
auto_sold_ewm == seasonal_monthday_auto_sold_ewm                                     0.31250
auto_sold_ewm == seasonal_weekday_seasonal_monthday_auto_sold_ewm                    0.00000
auto_sold_ewm == seasonal_auto_sold_ewm                                              0.00000
seasonal_weekday_auto_sold_ewm == seasonal_monthday_auto_sold_ewm                    0.00000
seasonal_weekday_auto_sold_ewm == seasonal_weekday_seasonal_monthday_auto_sold_ewm   0.09844
seasonal_weekday_auto_sold_ewm == seasonal_auto_sold_ewm                             0.00249
seasonal_monthday_auto_sold_ewm == seasonal_weekday_seasonal_monthday_auto_sold_ewm  0.00000
seasonal_monthday_auto_sold_ewm == seasonal_auto_sold_ewm                            0.00000
seasonal_weekday_se

In [92]:
# NEGATIVE STAT > FIRST MODEL PERFORMS BETTER
# POSITIVE STAT > SECOND MODEL PERFORMS BETTER

In [101]:
res

{'seasonal': {'auto_sold_ewm == seasonal_weekday_auto_sold_ewm': (0.0,
   22.818979914211234),
  'auto_sold_ewm == seasonal_monthday_auto_sold_ewm': (0.3125,
   1.0136806408021153),
  'auto_sold_ewm == seasonal_weekday_seasonal_monthday_auto_sold_ewm': (0.0,
   21.38830242441285),
  'auto_sold_ewm == seasonal_auto_sold_ewm': (0.0, 47.687629213826646),
  'seasonal_weekday_auto_sold_ewm == seasonal_monthday_auto_sold_ewm': (0.0,
   -9.636306974031259),
  'seasonal_weekday_auto_sold_ewm == seasonal_weekday_seasonal_monthday_auto_sold_ewm': (0.09844,
   1.663606342479339),
  'seasonal_weekday_auto_sold_ewm == seasonal_auto_sold_ewm': (0.00249,
   3.080907919417292),
  'seasonal_monthday_auto_sold_ewm == seasonal_weekday_seasonal_monthday_auto_sold_ewm': (0.0,
   14.83588007875148),
  'seasonal_monthday_auto_sold_ewm == seasonal_auto_sold_ewm': (0.0,
   18.537154759876827),
  'seasonal_weekday_seasonal_monthday_auto_sold_ewm == seasonal_auto_sold_ewm': (0.00017,
   3.8724022000049723)},
 's